In [ ]:
import torch
import torch.nn as nn

class Hamiltonian(nn.Module):
    def __init__(self, dim):
        super(Hamiltonian, self).__init__()
        self.dim = dim
        self.position = nn.Parameter(torch.randn(dim))
        self.momentum = nn.Parameter(torch.randn(dim))

    def forward(self, timesteps, step_size):
        # Define the Hamiltonian dynamics using leapfrog integrator
        for i in range(timesteps):
            # Update momentum
            self.momentum -= 0.5 * step_size * self.grad_potential(self.position)
            # Update position
            self.position += step_size * self.momentum
            # Update momentum again
            self.momentum -= 0.5 * step_size * self.grad_potential(self.position)

    def grad_potential(self, x):
        # Define the gradient of the potential energy function here
        return torch.autograd.grad(self.potential(x), x)[0]

    def potential(self, x):
        # Define the potential energy function here
        return torch.sum(x ** 2) / 2

# Example usage:
hamiltonian = Hamiltonian(10)
hamiltonian.forward(100, 0.01)

In [ ]:
import torch
import torch.nn.functional as F

def compute_elbo(model, x, num_samples=10):
    """
    计算未校正的哈密顿动力学模型下的ELBO
    
    参数：
        model: 未校正的哈密顿动力学模型
        x: 输入数据，大小为[batch_size, input_size]
        num_samples: 采样数量
    
    返回：
        elbo: ELBO（Evidence Lower Bound）
    """
    
    batch_size, input_size = x.size()

    # 从后验分布q(z|x)中采样num_samples个样本
    z_samples = []
    for i in range(num_samples):
        z_q, _ = model.q_z(x)
        z_samples.append(z_q)

    # 将样本堆叠成张量
    z_samples = torch.stack(z_samples)

    # 计算解码器p(x|z)的对数似然
    x_logits = model.p_x(z_samples).view(num_samples, batch_size, -1)
    log_likelihood = F.log_softmax(x_logits, dim=-1).sum(-1).mean(0)

    # 计算后验分布q(z|x)与先验分布p(z)之间的KL散度
    z_p = model.sample_prior(num_samples)
    kl_divergence = torch.distributions.kl_divergence(
        torch.distributions.Normal(z_samples.mean(0), z_samples.std(0)),
        torch.distributions.Normal(z_p, torch.ones_like(z_p))
    ).sum(-1).mean(0)

    # 计算ELBO
    elbo = log_likelihood - kl_divergence

    return -elbo  # 返回负数，因为我们使用优化器最小化损失函数